In [ ]:
import numpy as np
from sklearn.datasets import make_friedman2, make_regression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.datasets import load_boston
from sklearn.feature_selection import SelectKBest, f_classif

In [ ]:
data = pd.read_csv('data/dataset.csv')
y = data[['label']]
X = data.drop(['label'], axis=1)
X.head()

In [ ]:
X_train_pre = pd.read_csv('data/X_train_pre.csv')
X_test_pre = pd.read_csv('data/X_test_pre.csv')
y_train_pre = pd.read_csv('data/y_train_pre.csv')
y_test_pre = pd.read_csv('data/y_test_pre.csv')
X_pre = pd.concat([X_train_pre, X_test_pre])
y_pre = pd.concat([y_train_pre, y_test_pre])

In [ ]:
plt.figure(figsize=(12,10))
cor = data.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
# Find k best features
selector = SelectKBest(f_classif, k=10)

selected_data = selector.fit_transform(X, y)
selected_features = pd.DataFrame(selector.inverse_transform(selected_data), 
                                 index=data.index, 
                                 columns=X.columns)

selected_columns = selected_features.columns[selected_features.var() != 0]
selected_features[selected_columns].head()

In [ ]:
# Find k best features
selector = SelectKBest(f_classif, k=10)

selected_data = selector.fit_transform(X_pre, y_pre)
selected_features = pd.DataFrame(selector.inverse_transform(selected_data), 
                                 index=data.index, 
                                 columns=X_pre.columns)

selected_columns = selected_features.columns[selected_features.var() != 0]
selected_features[selected_columns].head()

In [ ]:
data = pd.read_csv('data/data_test.csv')

In [ ]:
df1 = data
# removing rows with more than % missing values
limitPer = len(df1.columns) * 0.5
df2 = df1.dropna(thresh=limitPer, axis=0) # thresh: Require that many non-NA values.
print("Number of Missing Columns: {}".format(limitPer))
print("Shape of Data Frame #1: {}".format(df1.shape))
print("Shape of Data Frame #2: {}".format(df2.shape))

In [ ]:
data['PHOS'].max()